In [ ]:
def hexagon():
    from bpy import data as D, context as C
    import bmesh

    mesh_data = D.meshes.new("Hex.Grid")

    bm = bmesh.new()
    bmesh.ops.create_circle(
        bm,
        segments=6,
        radius=1,
        cap_ends=True)
    bm.to_mesh(mesh_data)
    bm.clear()
    bm.free()

    mesh_obj = D.objects.new(mesh_data.name, mesh_data)
    C.scene.collection.objects.link(mesh_obj)
    
    mod=mesh_obj.modifiers.new(name="solid",type="SOLIDIFY")
    mod.offset=1
    mod.thickness=1
    
    return mesh_obj

In [ ]:
#smaller class and more classes
#keep it simple
class AddHandler:
    def __init__(self,size=2,li=None):
        if li is None:li=[]

        
        self.size=size
        self.li=li
        self.add_kwargs=dict(align='CURSOR', location=(0, 0, 0))
        
    def add_to_list(self,thing,li=None):
        if li is None:
            assert self.li is not None
            li=self.li
        i=self.add(thing)
        li.append(i)
        return i
        
    def add(self,thing):
        match thing:
            case "axes":
                bpy.ops.object.empty_add(type='PLAIN_AXES', **self.add_kwargs)
            case "cube":
                bpy.ops.mesh.primitive_cube_add(size=self.size, **self.add_kwargs)
            case "arma"|"armature":
                bpy.ops.object.armature_add()
            case "hex"|"hexagon"|"hexa":
                return hexagon_pillar()
            case "cyli"|"cylinder":
                bpy.ops.mesh.primitive_cylinder_add(radius=self.size,depth=self.size)
            case _:
                raise NotImplementedError
        return context.object

In [1]:
def give_axes_cylinder():
    """add an cylinder to empty axes"""

def insert_axes_links(child):
    a=AddHandler()
    axes=a.add("axes")
    # axes.parent=parent
    
    
    child.parent=axes
    del a
    return axes

In [ ]:
def property_handler(thing,location=None,scale=None,rotation=None,
                    change=None):
    if location is not None:
        if change: location=[i*ii for i,ii in zip(location,change)]
        thing.location=location
    if rotation is not None:
        import math
        if change: rotation=[i*ii for i,ii in zip(rotation,change)]
        thing.rotation_euler =tuple(math.radians(i) for i in rotation)
    if scale is not None:thing.scale=scale


In [ ]:
def view_selected():
    for area in bpy.context.screen.areas:
        if area.type == 'VIEW_3D':
            ctx = bpy.context.copy()
            ctx['area'] = area
            ctx['region'] = area.regions[-1]
            with context.temp_override(**ctx):
                bpy.ops.view3d.view_selected()   

In [8]:
def copy_ob(ob, parent,  collection=bpy.context.collection):
    # copy ob
    copy = ob.copy()
    copy.parent = parent
    copy.matrix_parent_inverse = ob.matrix_parent_inverse.copy()
    # copy particle settings
    for ps in copy.particle_systems:
        ps.settings = ps.settings.copy()
    collection.objects.link(copy)
    return copy
    
def tree_copy(ob, parent, levels=3):
    def recurse(ob, parent, depth):
        if depth > levels: 
            return
        copy = copy_ob(ob, parent)
        
        for child in ob.children:
            recurse(child, copy, depth + 1)
        return copy
    return recurse(ob, ob.parent, 0)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
import math,mathutils

In [ ]:
def remove_all_mods(obj):
    for i in obj.modifiers:
        bpy.ops.object.modifier_remove(modifier=i.name)

In [23]:
def new_material(m="m"):
    mat = bpy.data.materials.get(m)
    if mat is None:
        mat = bpy.data.materials.new(name=m)
    mat.use_nodes = True
    return mat
def give_material(x,m="g"):
    m=new_material(m)
    x.active_material=m
    return m

In [ ]:
def show(img):
    from matplotlib import pyplot as plt
    plt.imshow(img)
    plt.title('my picture')
    plt.show()
    

In [27]:
bpy.context.preferences.view.show_tooltips=True
bpy.context.preferences.view.show_tooltips_python=True
bpy.context.preferences.view.show_developer_ui=True